# Modelado Machine Learning

## Pruebas Saber Pro Colombia

El siguiente proyecto consiste en el uso de un modelo de Machine Learning, para poder predecir el desempeño de diferentes estudiantes de educación superior de Colombia en las Pruebas Saber Pro. Para el entrenamiento del modelo se utilizan diferentes variables o atributos, entre los cuales se encuentran:

- Información socieconómica: Características socieconómicas del estudiante, como su estrato, educación de sus padres, entre otras.

- Información institucional: Características asociadas a las instituciones educativas de donde provienen los estudiantes.

- Información estadística: Describe algunos coeficientes que equipos de estudio han desarrollado que podría ayudar a la clasificación.

## 1. Descarga de datos desde Kaggle

Realizamos la descarga de los archivos de entrenamiento y prueba del modelo, con el fin de realizar las predicciones necesarias para el desempeño (rendimiento global) de los estudiantes de la base de datos de prueba.

In [5]:
import os
os.environ["KAGGLE_CONFIG_DIR"] = "."

!chmod 600 ./kaggle.json
!kaggle competitions download -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia
!unzip -o "udea*.zip" > /dev/null
!ls

  0% 0.00/29.9M [00:00<?, ?B/s]
100% 29.9M/29.9M [00:00<00:00, 1.24GB/s]
kaggle.json		test.csv
sample_data		train.csv
submission_example.csv	udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip


In [6]:
!wc *.csv

   296787    296787   4716673 submission_example.csv
   296787   4565553  59185238 test.csv
   692501  10666231 143732437 train.csv
  1286075  15528571 207634348 total


## 2. Importación de librerías y funciones auxiliares

Importamos las librerías base de cálculos numéricos de python y las librerías de machine learning necesarias para implementar el modelo.

In [7]:
import pandas as pd
import numpy as np

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.metrics import accuracy_score, classification_report

train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

target_col = "RENDIMIENTO_GLOBAL"
id_col = "ID"

y = train[target_col]
X = train.drop(columns=[target_col])
feature_cols = [c for c in X.columns if c != id_col]

numeric_features = X[feature_cols].select_dtypes(include=["number"]).columns.tolist()
categorical_features = X[feature_cols].select_dtypes(exclude=["number"]).columns.tolist()

print("Columnas numéricas:", len(numeric_features))
print("Columnas categóricas:", len(categorical_features))

Columnas numéricas: 5
Columnas categóricas: 14


## 3. Construcción de un pipeline genérico de preprocesamiento
        
Creamos una función de preprocesamiento de los datos, con el fin de:

* Ingresar valores faltantes (mediana para numéricas, moda para categóricas).
* Estandarización de variables numéricas.
* Codificación *one‑hot* de categorías.

In [8]:
from sklearn.base import clone

def build_preprocessor():
    numeric_transformer = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler())
    ])

    categorical_transformer = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", OneHotEncoder(handle_unknown="ignore"))
    ])

    preprocessor = ColumnTransformer(
        transformers=[
            ("num", numeric_transformer, numeric_features),
            ("cat", categorical_transformer, categorical_features)
        ]
    )
    return preprocessor

def make_pipeline(estimator):
    return Pipeline(steps=[
        ("preprocessor", build_preprocessor()),
        ("classifier", estimator)
    ])

## 4. Definición del modelo

Vamos a utilizar el modelo **Logistic Regression** de Machine Learning, aplicado a clasificación multinomial y con una cantidad de iteraciones máxima de 200. Adicional a esto, realizamos una validación cruzada por medio de la función **StratifiedKFold** que nos garantiza que los datos queden debidamente balanceados.

In [43]:
model = {
    "LogisticRegression": LogisticRegression(
        max_iter=200,
        multi_class="multinomial",
        n_jobs=-1
    )
}

cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
cv_results = []

for name, est in model.items():
    pipe = make_pipeline(est)
    scores = cross_val_score(pipe, X[feature_cols], y, cv=cv, scoring="accuracy")
    cv_results.append({
        "Modelo": name,
        "Exactitud (accuracy)": scores.mean(),
        "Desviación estándar": scores.std()
    })
    print(f"{name}: {scores.mean():.4f} +/- {scores.std():.4f}")

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and wi

LogisticRegression: 0.4264 +/- 0.0014


Veamos qué datos obtuvimos del entrenamiento para la exactitud (accuracy) y la desviación estándar.

In [44]:
cv_results_df = pd.DataFrame(cv_results).sort_values("Exactitud (accuracy)", ascending=False)
cv_results_df

,Modelo,Exactitud (accuracy),Desviación estándar
0,LogisticRegression,0.426365,0.001433


## 5. Entrenamiento del modelo y evaluación hold‑out

Realizamos el entrenamiento del modelo con ayuda de los datos del archivo **train.csv**

In [47]:
model_name = cv_results_df.iloc[0]["Modelo"]
print("Modelo seleccionado:", model_name)

best_estimator = clone(model[model_name])
final_pipeline = make_pipeline(best_estimator)

X_train, X_valid, y_train, y_valid = train_test_split(
    X[feature_cols], y, test_size=0.2, random_state=42, stratify=y
)

final_pipeline.fit(X_train, y_train)
y_pred = final_pipeline.predict(X_valid)

print("Accuracy hold-out:", accuracy_score(y_valid, y_pred))
print(classification_report(y_valid, y_pred))

Modelo seleccionado: LogisticRegression


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Accuracy hold-out: 0.42524909747292416
              precision    recall  f1-score   support

        alto       0.53      0.63      0.57     35124
        bajo       0.45      0.56      0.50     34597
  medio-alto       0.32      0.25      0.28     34324
  medio-bajo       0.33      0.25      0.28     34455

    accuracy                           0.43    138500
   macro avg       0.41      0.42      0.41    138500
weighted avg       0.41      0.43      0.41    138500



## 6. Entrenamiento con todos los datos y generación de envío a Kaggle

Realizamos un entrenamiento final con todos los datos de **train.csv**, y así mismo realizamos la aplicación del modelo a los datos de prueba para poder obtener las predicciones buscadas.

In [48]:
# Re-entrenar con todos los datos
final_pipeline.fit(X[feature_cols], y)

X_test = test[feature_cols].copy()
test_pred = final_pipeline.predict(X_test)

submission = pd.DataFrame({
    "ID": test[id_col],
    "RENDIMIENTO_GLOBAL": test_pred
})

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


In [51]:
submission.to_csv("submission.csv", index=False) #Convertir el dataframe obtenido en archivo .csv
submission.head()

,ID,RENDIMIENTO_GLOBAL
0,550236,bajo
1,98545,bajo
2,499179,alto
3,782980,bajo
4,785185,bajo


In [50]:
print(submission['RENDIMIENTO_GLOBAL'].value_counts())

RENDIMIENTO_GLOBAL
bajo          93025
alto          89660
medio-alto    57261
medio-bajo    56840
Name: count, dtype: int64


## 7. Envío de los resultados a la competencia en Kaggle

Una vez tenemos el archivo .csv con los resultados, realizamos la entrega de las predicciones obtenidas para cada uno de los estudiantes del archivo de prueba a la competencia en Kaggle para calcular el **accuracy** resultante.

In [52]:
!head submission.csv
!kaggle competitions submit -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia -f submission.csv -m "Prueba final - LogisticRegression"

ID,RENDIMIENTO_GLOBAL
550236,bajo
98545,bajo
499179,alto
782980,bajo
785185,bajo
58495,bajo
705444,alto
557548,alto
519909,bajo
100% 4.02M/4.02M [00:00<00:00, 6.06MB/s]
Successfully submitted to UDEA/ai4eng 20252 - Pruebas Saber Pro Colombia